# Авторизуемся

In [1]:
import test

In [2]:
KEY_FILE_LOCATION = 'path'
anal_cred = test.get_credentials(KEY_FILE_LOCATION)

# А теперь построим наш первый репорт

In [ ]:
# anal_cred - это инстанс класса Resource

In [3]:
type(anal_cred)

googleapiclient.discovery.Resource

Будем пользоваться методом reports.batchGet объекта Resource<br>
Ссылка на документацию: https://developers.google.com/analytics/devguides/reporting/core/v4/rest/v4/reports/batchGet#top_of_page <br><br>

Для начала создадим дефолтное тело запроса <br>
Ссылки на документацию того как это делать:<br>
https://developers.google.com/analytics/devguides/reporting/core/v4/rest/v4/reports/batchGet#request-body <br>
https://developers.google.com/analytics/devguides/reporting/core/v4/rest/v4/reports/batchGet#ReportRequest <br>
сами поля:<br>
https://developers.google.com/analytics/devguides/reporting/core/dimsmets <br><br>

По сути в объекте Request Body внутри должен содержаться как минимум один ReportRequest object. Более подробно можно почитать в документации. <br>

И да, самое время записать в константу наш VIEW ID

In [1]:
VIEW_ID = 'id'
body={
    'reportRequests': [
        {
          'viewId': VIEW_ID,
          'dateRanges': [{'startDate': '7daysAgo', 'endDate': 'today'}],
          'metrics': [{'expression': 'ga:sessions'},{'expression': 'ga:users'}],
          'dimensions': [{'name': 'ga:country'},{'name': 'ga:sourceMedium'}]
}]}

## А теперь запрашиваем данные
И получаем в ответ Responce body <br>
https://developers.google.com/analytics/devguides/reporting/core/v4/rest/v4/reports/batchGet#response-body <br>
Внутри которого Report's <br>
https://developers.google.com/analytics/devguides/reporting/core/v4/rest/v4/reports/batchGet#Report <br>

In [4]:
responce = anal_cred.reports().batchGet(body=body).execute()

In [26]:
responce['reports'][0].keys()

dict_keys(['columnHeader', 'data'])

In [5]:
report = responce['reports'][0]

In [39]:
report

{'columnHeader': {'dimensions': ['ga:country', 'ga:sourceMedium'],
  'metricHeader': {'metricHeaderEntries': [{'name': 'ga:sessions',
     'type': 'INTEGER'},
    {'name': 'ga:users', 'type': 'INTEGER'}]}},
 'data': {'rows': [{'dimensions': ['Belarus', '(direct) / (none)'],
    'metrics': [{'values': ['1', '1']}]},
   {'dimensions': ['Belarus', 'google / organic'],
    'metrics': [{'values': ['5', '5']}]},
   {'dimensions': ['China', '(direct) / (none)'],
    'metrics': [{'values': ['1', '1']}]},
   {'dimensions': ['Cyprus', 'google / organic'],
    'metrics': [{'values': ['1', '1']}]},
   {'dimensions': ['France', '(direct) / (none)'],
    'metrics': [{'values': ['14', '14']}]},
   {'dimensions': ['Kazakhstan', '(direct) / (none)'],
    'metrics': [{'values': ['1', '1']}]},
   {'dimensions': ['Netherlands', 'google / organic'],
    'metrics': [{'values': ['2', '2']}]},
   {'dimensions': ['New Zealand', 'away.vk.com / referral'],
    'metrics': [{'values': ['1', '1']}]},
   {'dimension

### В целом чуть больше чем полностью неудобная форма представления, попробуем это изменить

По сути предсталвение данных имеет на самом деле табличную форму, но можно с этим всем поиграться и попоробуем имитировать колоночную базу данных, только без сортировки, а то слишком заморочено получится<br>

В чем прикол колоночных баз данных можно почитать вот тут:<br>
https://ruhighload.com/Колоночные+базы+данных

In [34]:
columnHeader = report.get('columnHeader', {})
data = {}
dimensions = []
metrics = []

for dimension in columnHeader.get('dimensions',[]):
    name = dimension.split(':')[1]
    dimensions.append(name)
    data[name] = []
for metric in columnHeader['metricHeader']['metricHeaderEntries']:
    name = metric['name'].split(':')[1]
    metrics.append(name)
    data[name] = []

# dimensions may not be, they are not required in request
if dimensions:
    for row in report['data']['rows']:
        for i,value in enumerate(row['dimensions']):
            data[dimensions[i]].append(value)
        for i,value in enumerate(row['metrics'][0]['values']):
            data[metrics[i]].append(value)
else:
    for row in report['data']['rows']:
        for i,value in enumerate(row['metrics'][0]['values']):
            data[metrics[i]].append(value)

### Проверим, все ли норм

In [37]:
sum1,sum2 = 0, 0 
cnt_rows = 0
for sess,us in zip(data['sessions'],data['users']):
    sum1, sum2 = sum1 + int(sess), sum2 + int(us)
    cnt_rows += 1

In [38]:
sum1, sum2, cnt_rows

(187, 159, 33)

### Но все равно не очень удобно, хотя если нужна скорость этот вариант близок к оптимальному
### Преобразуем в DataFrame

In [42]:
import pandas as pd

In [43]:
df = pd.DataFrame().from_dict(data)

In [45]:
df['sessions'] = pd.to_numeric(df['sessions'])
df['users'] = pd.to_numeric(df['users'])

In [47]:
# все норм
df['sessions'].sum() , df['users'].sum(), df.shape[0]

(187, 159, 33)

### И все это тоже закинем в модуль